Python libraries.

In [1]:
import praw
import pandas as pd

Configure PRAW library connection to Reddit API and bind instance to "reddit" variable.

In [2]:
reddit = praw.Reddit(client_id = "OdtsOvr97jHWbkQt6_habg", #peronal use script ID
                    client_secret = "xBTlIHnYgXrANCTDLP0ZCDupPfUNbg", #secret token
                    usernme = "rock4777", #profile username
                    password = "geckoman1995", #profile password
                    user_agent = "USERAGENT")

Create empty lists to be filled with post data.

In [3]:
id_list = []
title_list = []
upvote_ratio_list = []
score_list = []

Loop through posts and fill lists.

In [4]:

for submission in reddit.subreddit("wallstreetbets").hot(limit = 10000):
    id_list.append(submission.id)
    title_list.append(submission.title)
    score_list.append(submission.score)
    upvote_ratio_list.append(submission.upvote_ratio)
    
print('total', len(title_list), 'posts has been scraped')

total 699 posts has been scraped


Create data frame by combining all lists.

In [5]:
df = pd.DataFrame({'ID':id_list, 
                   'Title':title_list,
                   'Upvote_Count':score_list,
                   'Upvote_Ratio':upvote_ratio_list,
                  })
# df.to_csv('reddit_dataset.csv', index = False)